# [Research Task - Create visuals for PUC 99314.11 leg report](https://github.com/cal-itp/data-analyses/issues/1656)
1. line graph of each metric (UPT, VRM, PMT) by agency
- x-axis is year
- y-axis is metric
- each line is an agency
- dotted line is average metric for all agencies in the year

2. line graph of each metric, by district
- similar to above
- each line is a district
- dotted line is average metrics for all districts the year

3. line graph of each metric, by mode
- similar to above
- each line is a mode
- dotter line is average metric for all modes in the year

Maybe try a box plot to show min/max/average for each metric?

## NTD Policy Manual for collecting UPT and PMT

### NTD Full Reporting Policy Manual 
However, FTA recognizes that certain statistics are challenging to collect and can drastically increase the reporting burden for transit agencies. To assist reporters who would find conducting 100 percent count burdensome, `transit agencies may estimate Unlinked Passenger Trips (UPT) and PMT through sampling`. The NTD provides a sampling method and sampling guidance on the NTD website.

### NTD Full Reporting Policy Manual & NTD Reduced Reporting Polict Manual
Collecting Service Consumed Data Transit agencies must report actual data on the Annual Report for all service data except UPT and PMT. `Only Full Reporters report PMT data to the NTD.` For these two data points, agencies may provide an estimate but only if the actual 100 percent data are not reliably collected and routinely processed.



In [1]:
import altair as alt
import pandas as pd
from calitp_data_analysis.sql import get_engine, to_snakecase

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

## warehouse query

In [ ]:
metric_list = [
    "pmt",
    "upt",
    "vrh",
    # "opexp_total" # not needed for this project
]

# empty list for appending DFs
df_list = []

db_engine = get_engine()

with db_engine.connect() as connection:
    for metric in metric_list:
        query = f"""
        SELECT
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          service,
          reporter_type,
          SUM({metric}) AS total_{metric},
        FROM
          `cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_{metric}`
        WHERE
          source_state = "CA"
          AND year >= 2018
        GROUP BY
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          service,
          reporter_type
        """
        # create df
        metric = pd.read_sql(query, connection)

        # append df to list
        df_list.append(metric)

# unpack list into separate DFs
ntd_pmt, ntd_upt, ntd_vrh = df_list

In [ ]:
# get districts for ntd ID

with db_engine.connect() as connection:
    for metric in metric_list:
        query = f"""
        SELECT
          `mart_transit_database.dim_organizations`.`key` AS `key`,
          `mart_transit_database.dim_organizations`.`source_record_id` AS `source_record_id`,
          `mart_transit_database.dim_organizations`.`name` AS `name`,
          `mart_transit_database.dim_organizations`.`ntd_id_2022` AS `ntd_id_2022`,
          `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_name` AS `county`,
          `Dim_County_Geography___County_Geography_Key`.`caltrans_district` AS `caltrans_district`
        FROM
          `mart_transit_database.dim_organizations`

        LEFT JOIN `mart_transit_database.bridge_organizations_x_headquarters_county_geography` AS `Bridge_Organizations_X_Headquarters_County_Geography___Key` ON `mart_transit_database.dim_organizations`.`key` = `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`organization_key`
          LEFT JOIN `mart_transit_database.dim_county_geography` AS `Dim_County_Geography___County_Geography_Key` ON `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_key` = `Dim_County_Geography___County_Geography_Key`.`key`
        WHERE
          (
            `mart_transit_database.dim_organizations`.`_is_current` = TRUE
          )

           AND (
            `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NOT NULL
          )
          AND (
            (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` <> ''
            )

            OR (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NULL
            )
          )
          AND (
            `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`_is_current` = TRUE
          )
          AND (
            `Dim_County_Geography___County_Geography_Key`.`_is_current` = TRUE
          )
        """
        # create df
        ntd_id_x_district = pd.read_sql(query, connection)
        
ntd_id_x_district["caltrans_district"] = ntd_id_x_district["caltrans_district"].astype("str")

In [ ]:
ntd_id_x_district.info()

In [ ]:
merge_on_col = [
    "ntd_id",
    "year",
    "source_agency",
    "agency_status",
    "primary_uza_name",
    "uza_population",
    "uza_area_sq_miles",
    "mode",
    "service",
    "reporter_type",
]

merge_1 = ntd_vrh.merge(ntd_upt, on=merge_on_col, how="inner")
# merge_2 = merge_1.merge(ntd_vrh, on=merge_on_col, how = "inner")

ntd_metrics_merge = merge_1.merge(ntd_pmt, on=merge_on_col, how="inner")

In [ ]:
ntd_metrics_merge.info()

## data from other report

In [ ]:
gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
ntd_name = "ntd_operator_data_18_23.parquet"

ntd_all_metrics = pd.read_parquet(f"{gcs_path}{ntd_name}")

## compare datasets

In [ ]:
display(
    ntd_all_metrics.info(), ntd_metrics_merge.info()  # mode/service is aggregated up
)

In [ ]:
display(
    ntd_all_metrics["ntd_id"].nunique()
    == ntd_metrics_merge["ntd_id"].nunique(),  # TRUE, same count of unique values
    set(ntd_all_metrics["ntd_id"].unique())
    == set(ntd_metrics_merge["ntd_id"].unique()),  # TRUE, same unique NTD_IDs
)

In [ ]:
metric_cols = ["total_upt", "total_vrh", "total_upt"]

for metric in metric_cols:
    print(
        ntd_all_metrics[metric].sum() == ntd_metrics_merge[metric].sum()
    )  # TRUE sum of each metrics are equal

## merge in the district numbers to ntd_metric_merge

In [ ]:
ntd_metrics_merge = ntd_metrics_merge.merge(
    ntd_id_x_district[["ntd_id_2022","county","caltrans_district"]],
    left_on = "ntd_id",
    right_on = "ntd_id_2022",
    how="left",
    indicator=True
)

In [ ]:
ntd_metrics_merge.info()

In [ ]:
ntd_metrics_merge[ntd_metrics_merge["_merge"]=="left_only"]["source_agency"].value_counts().head()

In [ ]:
ntd_metrics_merge[ntd_metrics_merge["caltrans_district"].isna()].head()

### save out data


In [2]:
gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
# ntd_metrics_merge.to_parquet(f"{gcs_path}puc_analysis_data.parquet")

In [3]:
ntd_metrics_merge = pd.read_parquet(f"{gcs_path}puc_analysis_data.parquet")
ntd_metrics_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4152 entries, 0 to 4151
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   ntd_id             4002 non-null   object  
 1   source_agency      4152 non-null   object  
 2   agency_status      4152 non-null   object  
 3   primary_uza_name   4152 non-null   object  
 4   uza_population     4152 non-null   int64   
 5   uza_area_sq_miles  4152 non-null   float64 
 6   year               4152 non-null   int64   
 7   mode               4152 non-null   object  
 8   service            4152 non-null   object  
 9   reporter_type      4152 non-null   object  
 10  total_vrh          2623 non-null   float64 
 11  total_upt          2623 non-null   float64 
 12  total_pmt          2623 non-null   float64 
 13  ntd_id_2022        3798 non-null   object  
 14  county             3798 non-null   object  
 15  caltrans_district  3798 non-null   object  
 16  _merge

## everything matches, moving with `ntd_metrics_merge` since its has mode/service`

In [4]:
# melt big DF so all columns are under 1 column.
group_list = [
    "source_agency",
    "year",
    "ntd_id",
    "reporter_type",
    "caltrans_district"
]
value_cols = ["total_upt", "total_vrh", "total_pmt"]

melt = pd.melt(
    ntd_metrics_merge,
    id_vars=group_list,
    value_vars=value_cols,
    var_name="metric",
    value_name="metric_value",
    ignore_index=True,
)

In [5]:
melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12456 entries, 0 to 12455
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   source_agency      12456 non-null  object 
 1   year               12456 non-null  int64  
 2   ntd_id             12006 non-null  object 
 3   reporter_type      12456 non-null  object 
 4   caltrans_district  11394 non-null  object 
 5   metric             12456 non-null  object 
 6   metric_value       7869 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 681.3+ KB


## Group aggregations

In [6]:
# What does group/agg the melted DF look like?
vrh_total = (
    melt[melt["metric"] == "total_vrh"]
    .groupby(group_list)["metric_value"]
    .sum()
    .reset_index()
).rename(columns={"metric_value": "total_vrh"})

upt_total = (
    melt[melt["metric"] == "total_upt"]
    .groupby(group_list)["metric_value"]
    .sum()
    .reset_index()
).rename(columns={"metric_value": "total_upt"})

passenger_total = (
    melt[melt["metric"] == "total_pmt"]
    .groupby(group_list)["metric_value"]
    .sum()
    .reset_index()
).rename(columns={"metric_value": "total_pmt"})

yearly_totals = (
    ntd_metrics_merge.groupby(["year"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
) 

district_totals = (
    ntd_metrics_merge.groupby(["caltrans_district","year"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

mode_totals = (
    ntd_metrics_merge.groupby(["mode","year"])
    .agg({"total_upt": "sum", "total_vrh": "sum", "total_pmt": "sum"})
    .reset_index()
)

In [ ]:
all_totals = [vrh_total, upt_total, passenger_total]

for group in all_totals:
    print(group.info(), group["ntd_id"].nunique())


In [ ]:
# how many rows have zero PMT?
len(passenger_total[passenger_total["total_pmt"] == 0])

In [ ]:
def make_chart(data, col):
    chart = (
        alt.Chart(data)
        .mark_line(point=True)
        .encode(
            x=alt.X("year:N", axis=alt.Axis(format="$.2f")),
            y=f"{col}",
            color=alt.Color("ntd_id:N", title="Agency"),
            # row= "reporter_type",
            tooltip=["source_agency", col, "year"],
        )
        .properties(
            title=f"{col} per Reporter Type",
            height=200,
            width=600,
        )
        .resolve_scale(y="independent")
        .interactive()
    )

    return chart

## Overall Totals

### Line chart of each metric grand total per year, with mean 

In [ ]:
yearly_totals.columns

In [ ]:
for col in yearly_totals.columns[1:]:
    total_chart = (
        alt.Chart(yearly_totals)
        .mark_line(point=True)
        .encode(
            x="year:N",
            y=alt.Y(f"{col}:Q", title=f"{col}"),
            # color= alt.Color("mode:N",),
            # row= "reporter_type",
            tooltip=[col],
        )
        .properties(
            title=f"{col} per year",
            height=200,
            width=600,
        )
        .interactive()
    )

    # line for average
    line = (
        alt.Chart(yearly_totals)
        .mark_rule(color="red", strokeWidth=1, strokeDash=[10, 5], point=True)
        .encode(
            # x="year:N",
            y=alt.Y(f"mean({col}):Q"),
            tooltip=[f"mean({col}):Q"],
        )
    )

    display((total_chart + line))

### Boxplot of each metric grand total per year

In [ ]:
all_totals_dict = {
    "total_vrh": vrh_total,
    "total_upt": upt_total,
    "total_pmt": passenger_total,
}

In [ ]:
# Boxplot
# removing zero-values to see what happens
for col, df in all_totals_dict.items():
    box_plot = (
        alt.Chart(df[df[col] != 0])
        .mark_boxplot(extent="min-max")
        .encode(
            x="year:N",
            y=col,
            # row = "reporter_type",
            tooltip=["source_agency", col, "year"],
        )
        .interactive()
        .properties(title=col, height=200, width=1000)
    )

    display(
        f"Number of Agencies that reported zero {col}: {df[df[col]==0].ntd_id.nunique()}",
        box_plot.resolve_scale(y="independent"),
    )

## plot by district

In [ ]:
district_totals.columns[2:]

In [ ]:
# color by district
for col in district_totals.columns[2:]:
    total_chart = (
        alt.Chart(district_totals)
        .mark_line(point=True)
        .encode(
            x="year:N",
            y=alt.Y(col, title=f"{col}"),
            color= alt.Color("caltrans_district:N",),
            # row= "reporter_type",
            tooltip=["caltrans_district",col],
        )
        .properties(
            title=f"{col} by District per year",
            height=300,
            width=600,
        )
        .interactive()
    )

    # line for average
    line = (
        alt.Chart(district_totals)
        .mark_rule(color="red", strokeWidth=1, strokeDash=[10, 5], point=True)
        .encode(
            # x="year:N",
            y=alt.Y(f"mean({col}):Q"),
            tooltip=[f"mean({col}):Q"],
        )
    )

    display((total_chart + line))

In [ ]:
# Boxplot
# removing zero-values to see what happens
for col in district_totals.columns[2:]:
    box_plot = (
        alt.Chart(district_totals[district_totals[col] != 0])
        .mark_boxplot(extent="min-max")
        .encode(
            x="caltrans_district:N",
            y=col,
            # row = "reporter_type",
            tooltip=[col, "year"],
        )
        .interactive()
        .properties(title=f"Box Plot of {col} per district", height=200, width=1000)
    )

    display(
        # f"Number of Agencies that reported zero {col}: {district_totals[district_totals[col]==0].ntd_id.nunique()}",
        box_plot.resolve_scale(y="independent"),
    )

### Line chart of metrics by agency by year by reporter type

In [ ]:
for col, df in all_totals_dict.items():
    chart = (
        alt.Chart(df)
        .mark_line(point=True)
        .encode(
            x="year:N",
            y=f"{col}",
            color=alt.Color("ntd_id:N", title="Agency"),
            # row="reporter_type",
            tooltip=["source_agency", col, "year"],
        )
        .properties(
            title=f"{col} per Agency",
            height=200,
            width=600,
        )
        .resolve_scale(y="independent")
        .interactive()
    )

    # line for average
    # line = alt.Chart(df).mark_line(
    #     color="black",
    #     strokeWidth = 3,
    #     strokeDash =[10, 5],
    #     point=True
    # ).encode(
    #     # x="year:T",
    #     y= alt.Y(f"mean({col})", title = "Average"),
    #     tooltip=["year",f"mean({col})"]
    # )

    display(
        f"Number of Agencies that reported zero {col}: {df[df[col]==0].ntd_id.nunique()}",
        chart,
    )

## plot of metric by mode

In [ ]:
# color by district
for col in mode_totals.columns[2:]:
    total_chart = (
        alt.Chart(mode_totals)
        .mark_line(point=True)
        .encode(
            x="year:N",
            y=alt.Y(col, title=f"{col}"),
            color= alt.Color("mode:N",),
            # row= "reporter_type",
            tooltip=["mode",col],
        )
        .properties(
            title=f"{col} by Mode per year",
            height=300,
            width=600,
        )
        .interactive()
    )

    # line for average
    line = (
        alt.Chart(mode_totals)
        .mark_rule(color="red", strokeWidth=1, strokeDash=[10, 5], point=True)
        .encode(
            # x="year:N",
            y=alt.Y(f"mean({col}):Q"),
            tooltip=[f"mean({col}):Q"],
        )
    )

    display((total_chart + line))